Extracting skills from job description
Function will return nested dict (see NLP_skillNer_example)

!NOTE: The extraction is a very long process. 500 jobs can take multiple hours. 

There are errors in specific rows. Solved beneath "return None" and with a "for loop and continue" beneath
If you want the dictionnaires for each row as a column of the df, run cell below with the function "skills"

In [1]:
# imports SKILLNER
import spacy
from spacy.matcher import PhraseMatcher

# load default skills data base
from skillNer.general_params import SKILL_DB # EMSI skills database 
# import skill extractor
from skillNer.skill_extractor_class import SkillExtractor

import pandas as pd 

In [2]:
df_main = pd.read_csv("data/gsearch_cleaned.csv")

In [3]:
df_main.head()

,Unnamed: 0,index,title,company_name,location,via,description,extensions,schedule_type,work_from_home,...,python,sql,tableau,bi tool,power bi,aws,azure,excel,powerpoint,r
0,0,0,data analyst,Cisco Meraki,Anywhere,via linkedin,"as the leader in cloud-managed it, cisco merak...","['22 hours ago', 'Work from home', 'Full-time'...",Full-time,True,...,0,1,1,1,0,1,0,0,0,1
1,1,1,entry level - business data analyst (remote),Midwest Staffing,Anywhere,via linkedin,as a senior business analyst you will contribu...,"['53 minutes ago', 'Work from home', 'Full-tim...",Full-time,True,...,0,0,0,0,0,0,0,0,0,1
2,2,2,data analyst/researcher,"Amyx, Inc.",United States,via indeed,overview:\n\namyx is seeking to hire a data an...,"['3 hours ago', 'Full-time']",Full-time,True,...,0,1,0,0,0,1,1,1,0,1
3,3,3,data analyst/ai expert to help build a website...,Upwork,Anywhere,via upwork,i am looking for someone to help me build an a...,"['23 hours ago', 'Work from home', 'Contractor']",Contractor,True,...,0,0,0,0,0,0,0,1,0,1
4,4,4,data analyst,Bank Street College of Education,United States,via higher education recruitment consortium (h...,position vacancy – data analyst to support the...,"['20 hours ago', 'Full-time']",Full-time,True,...,0,0,0,0,0,0,0,1,1,1


In [3]:
# NLP + skillner
nlp = spacy.load("en_core_web_lg")
# init skill extractor
skill_extractor = SkillExtractor(nlp, SKILL_DB, PhraseMatcher)

loading full_matcher ...
loading abv_matcher ...
loading full_uni_matcher ...
loading low_form_matcher ...
loading token_matcher ...


In [ ]:
# #dictionnaires for each row
# def skills(text):
#     try:
#         skills_ex = skill_extractor.annotate(text)
#         return(skills_ex)
#     except Exception as e:
#         return None

# # applies the function to the range of rows specified by indexing
# df_main['skills_ex'] = df_main['description_cleaned'][0:5].apply(skills) 

If you want a list of dictionnaires run the loop

In [ ]:
# Workaround of skillners bugs by ignoring the errors
# looping through rows in df.description_cleaned
counter = 0
#The list that returns the indexes of the error "out of range" for [0:500] = [146, 171, 247, 249, 274, 285, 286, 355, 357, 438, 499]
index_list = [] 
skills_list = [] 


for id in df_main['description_cleaned'][29500:31000]: #specify the index range 
    try:
        skills_ex = skill_extractor.annotate(id)
        skills_list.append(skills_ex)
        counter = counter + 1 #ignores this when fails
        print(counter)
    except Exception as e:
        print(f"Error processing entry at counter {counter}: {str(e)}")
        index_list.append(counter) # list of indexes that werent processed
        counter = counter + 1
        continue

print(counter)
print(len(skills_list))
display(skills_list)
print(index_list)

In [ ]:
skills_list_df = pd.DataFrame(skills_list)
#skills_list_df.to_csv("data/skills_list_31500_32022.csv") # index_list [18,19]
#skills_list_df.to_csv("data/skills_list_31000_31500.csv") # index list empty
#skills_list_df.to_csv("data/skills_list_29500_31000.csv") #[2, 98, 103, 261, 262, 282, 311, 448, 526, 758, 759, 883, 911, 926, 943, 955, 963, 1000]

In [ ]:
#SKILLS = pd.read_csv("data/skills_list_31500_32022.csv")
#SKILLS = pd.read_csv("data/skills_list_31000_31500.csv")# genau angucken! position summary what you ii do working at ...
SKILLS = pd.read_csv("data/skills_list_29500_31000.csv")


Tokenization

In [4]:
tryme = pd.read_excel("data/skills_counts_jobs_20.xlsx")
tryme.head()

,skill,count,type
0,Analytics,3385,Hard Skill
1,Management,1949,Soft Skill
2,Communications,1899,Soft Skill
3,SQL (Programming Language),1780,Hard Skill
4,Tooling,1769,Hard Skill


In [ ]:
df_tokens = df_main.copy()
df_tokens.head()

In [8]:
#starting tokenization
tryme["skill"] = tryme["skill"].str.lower().head(50) # for top 50 skills # data collection

skills_list = tryme["skill"].head(50)

# create new columns for each skill
for skill in skills_list:
    df_tokens[skill] = df_tokens['description_cleaned'].apply(lambda x: 1 if skill.lower() in x.lower() else 0)

In [14]:
tryme["skill"] = tryme["skill"].drop(2).reset_index(drop=True)
tryme.head(15)

,skill,count,type
0,communications,3385,Hard Skill
1,sql (programming language),1949,Soft Skill
2,positivity,1899,Soft Skill
3,maintainability,1780,Hard Skill
4,operations,1769,Hard Skill
5,collaboration,1621,Soft Skill
6,disabilities,1448,Hard Skill
7,research,1385,Soft Skill
8,data quality,1191,Soft Skill
9,levelling,1131,Hard Skill


In [7]:
df_tokens[["analytics", "management", "communications", "sql (programming language)", "tooling"]].sum(axis=0).sort_values(ascending = False)


management                    18399
analytics                     16913
communications                 2624
tooling                         287
sql (programming language)        4
dtype: int64

In [32]:
display(df_tokens[["leveling", "description_cleaned"]])

KeyError: "['leveling'] not in index"

position vacancy – data analyst to support the ed center’s project portfolio, bank street education center (the ed center)summary position: bank street education center (the ed center), a division within bank street college of education (the college) created to support the college’s systems-level work with school districts, is currently seeking a data analyst to support projects across the ed... center’s portfolio. they will lead measurement planning as well as develop associated data collection tools to support their assigned projects.key relationships: the data analyst will report to the director of impact and work closely with project teams as well as the broader sspp team.responsibilities: in collaboration with their supervisors, their project teams, and broader sspp team, the data analyst will be responsible for the following:analytics:• collaborating with relevant team members to draft a measurement plan that clearly documents desired outcomes and goals as well as associated project metrics.• in collaboration with key stakeholders, creating data collection tools that capture data aligned to key project metrics as defined by project leads and in the measurement plan (e.g., tools to support teachers to measure the impact of inquiry cycles, surveys, observation tools, interviews).• analyzing key data points and prepare reports to share with internal and external audiences (e.g., team meetings, district teams, funder reporting and proposals, marketing materials, etc.).• collecting and tracking data around the quality of experiences of participants, and reflecting on this information to debrief learnings and surface implications on the project with project leaders.• building project tools (e.g., feedback forms, surveys, session reports, final report, etc.) and vetting internally.• troubleshooting technological support as it relates to data collection tools, as needed.• creating data visualizations for use by various stakeholders (teachers visualizing student-level data, principals visualizing school-level data, districts visualizing district-level data, and ed center team visualizing the progress and impact of our network).• supporting in building and maintaining data management structures and tools for short-term and long-term qualitative and quantitative data collection and analysis.• supporting project teams to deeply understand the district context by researching the history of the city/district and staying current on local topics.• engaging in self-initiated and job-provided professional development activities to stay current on matters related to education, data analysis, and project-related content.• meeting with the project team, sspp team, and ed center team leaders over the lifespan of the project to “step back” and check in on the progress of and make adjustments to the data collection of the project.• contributing to the sspp-wide analytics by informing a broader analytics strategy across the portfolio and serving as an analyst on other project teams as assigned; and performing related duties as assigned.project management & operations:• managing timelines for various analytics responsibilities and collaborating with the project manager to move forward respective workstreams.• attending professional learning engagements to support in data collection and implementation.• providing technical support for professional learning engagements, including managing logistical details and platform guidance in preparation of and during sessions.• supporting in maintaining a functional and supportive organizational system within the g:drive, files, documented systems and processes, etc.• performing related duties as assigned.qualifications:skills & knowledge:• background in research and education, with prior experience on research projects analyzing student-level data• expertise in interpreting and summarizing academic research as well as the ability to present information in multiple formats for diverse audiences.• strong problem solving skills with the ability to perform independently.• strong communications skills (email, writing, in-person).experiences:• at least 3-5 years of work experience• master’s degree in education, nonprofit policy, analytics, or related field• experience working with diverse and/or historically deprioritized student populations at the classroom- and/or school- levels.• experience with reporting and/or managing grant-funded projects and associated deliverables• ability to work effectively with data of varying levels of quality and validity; attention to detail and ability to identify issues and inconsistencies in data.• ability to manage complex and multiple projects• demonstrated ability to work collaboratively with demonstrated google suite, powerpoint, excel, google data studio, project management, and analytical skills diverse range of team members and external partners.• demonstrated ability to think strategically & execute on ideasmindsets:• an intersectional, anti-racist lens and strengths-based approach to data management, and team collaboration• a reflective learner’s stance• enthusiasm for learning and a willingness to self-initiate professional learning opportunities• strong work ethic and willingness to take initiativesalary range: $70,000 - $85,000additional information:• this position is contingent upon available funding.• please apply online by submitting a cover letter and resume for the position in the “job opportunities” section under the “about” tab on bank street’s website:https://www.bankstreet.edu/administrative-offices/human-resources/job-opportunities/• please address cover letters to:tracy fray-oliversenior associate vice president, bank street education centerbank street college610 west 112th streetnew york, ny 10025• please include where you came across this job posting in your cover letter.bank street college of education is an equal opportunity employer and does not discriminateon the basis of race, sexual orientation ethnic origin, sex, or disability in its employmentpolicies and other college administered programs

Skill list of jobs_20 dataset

In [ ]:
#loading data from sql to python
import sql_functions as sf

In [62]:
# load jobs data

schema = 'capstone_datacvpro'

# jobs_20 = sf.get_dataframe(f' SELECT * FROM {schema}.analysts_20')
# display(jobs_20)

df_skills = sf.get_dataframe(f' SELECT * FROM {schema}.skills_20') # skill table
display(df_skills)

,skill,count,type
0,Analytics,3385,Hard Skill
1,Management,1949,Soft Skill
2,Communications,1899,Soft Skill
3,SQL (Programming Language),1780,Hard Skill
4,Tooling,1769,Hard Skill
...,...,...,...
5618,Deconstruction,1,Hard Skill
5619,Motorcycles,1,Hard Skill
5620,Holistic Health,1,Hard Skill
5621,Body Work,1,Hard Skill


In [90]:
#df_skills= df_skills.drop(92).reset_index(drop=True)
# dropped skills: analytics, management, tooling, operations, disabilities, levelling,  equalization, activism, industrialization, job descriptions	, additives (34), governance, Hostile Work Environment, 
display(df_skills["count"].tail())

5605    1
5606    1
5607    1
5608    1
5609    1
Name: count, dtype: int64

In [104]:
df_top_skills = df_skills.query("count > 40").copy()
display(df_top_skills)

,skill,count,type
0,Communications,1899,Soft Skill
1,SQL (Programming Language),1780,Hard Skill
2,Positivity,1621,Soft Skill
3,Maintainability,1448,Hard Skill
4,Collaboration,1191,Soft Skill
...,...,...,...
507,Database Queries,41,Hard Skill
508,Good Documentation Practices,41,Hard Skill
509,Very Large Databases (VLDB),41,Hard Skill
510,Diagramming Software,41,Hard Skill


In [109]:
#df_top_skills
# Stemming: reduce words to their root
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

1. Search for word stem
2. Search for entity r, e
3. Remove () and insides, e.g., sql instead of sql (programming language)
4. key performance indicator or kpi?

In [110]:
#make a list of all skills
skill_list_top = df_top_skills.skill.tolist()

# #stammer
ps = PorterStemmer()
  
for w in skill_list_top:
    print(w, " : ", ps.stem(w))


Communications  :  commun
SQL (Programming Language)  :  sql (programming language)
Positivity  :  posit
Maintainability  :  maintain
Collaboration  :  collabor
Research  :  research
Data Quality  :  data qu
Presentations  :  present
Statistics  :  statist
Dashboard  :  dashboard
Planning  :  plan
Tableau (Business Intelligence Software)  :  tableau (business intelligence software)
Python (Programming Language)  :  python (programming language)
Component Object Model (COM)  :  component object model (com)
Integration  :  integr
Computer Science  :  computer sci
E (Programming Language)  :  e (programming language)
Decisiveness  :  decis
Innovation  :  innov
Writing  :  write
Personalization  :  person
Source Data  :  source data
Verbal Communication Skills  :  verbal communication skil
Consulting  :  consult
Data Visualization  :  data visu
Professionalism  :  profession
Data Analysis  :  data analysi
Accessioning  :  access
Minimum Data Set  :  minimum data set
Coloring  :  color
Exec

In [34]:
#starting tokenization
df_skills["skill"] = df_skills["skill"].str.lower().head(50) # for top 50 skills # data collection

skills_list = df_skills["skill"].head(50)

# create new columns for each skill
for skill in skills_list:
    df_tokens[skill] = df_tokens['description_cleaned'].apply(lambda x: 1 if skill.lower() in x.lower() else 0)

In [39]:
df_tokens.head()

0    False
1    False
2    False
3    False
4    False
Name: levelling, dtype: bool

In [38]:
df_tokens = df_tokens["levelling"] == True
#display(df_tokens[["levelling", "description_cleaned"]])

KeyError: 'levelling'

In [13]:
# uploading
# load to database

from dotenv import load_dotenv
load_dotenv()

# write dataset into database

# Import get_engine from sql_functions.py. You will need to restart your kernel and rerun at this point since we changed the module since we first imported it.
from sql_functions import get_engine
# create a variable called engine using the get_engine function
engine = get_engine()

import psycopg2

table_name = 'skills_20'
schema = 'capstone_datacvpro'

# Write records stored in a dataframe to SQL database
if engine!=None:
    try:
        df_skills.to_sql(name=table_name, # Name of SQL table variable
                        con=engine, # Engine or connection
                        schema=schema, # your class schema variable
                        if_exists='replace', # Drop the table before inserting new values 
                        index=False, # Write DataFrame index as a column
                        chunksize=5000, # Specify the number of rows in each batch to be written at a time
                        method='multi') # Pass multiple values in a single INSERT clause
        print(f"The {table_name} table was imported successfully.")
    # Error handling
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None
else:
    print('No engine')

name 'df_skills' is not defined
